# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: <Coloca el nombre de tu carrera aqui>** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 06**: Big Data Pipeline for Netflix data

**Fecha**: 7 de febrero de 2025

**Nombre del Estudiante**: JJAE

**Profesor**: Pablo Camarillo Ramirez

In [28]:
import findspark
findspark.init()

In [29]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions") \
    .master("spark://69ce0b83527f:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

# Netflix data

In [34]:
from JJAE.spark_utils import SparkUtils
netflix_schema = SparkUtils.generate_schema([
    ("show_id", "StringType"),
    ("type", "StringType"),
    ("title", "StringType"),
    ("director", "StringType"),
    ("country", "StringType"),
    ("date_added", "StringType"),
    ("release_year", "IntegerType"),
    ("rating", "StringType"),
    ("duration", "StringType"),
    ("listed_in", "StringType")
    ])

netflix_df = spark.read \
                .schema(netflix_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/netflix1.csv")

netflix_df.printSchema()

netflix_df.show(5, truncate=False)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)

+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|show_id|type   |title                           |director       |country      |date_added|release_year|rating|duration|listed_in                                                    |
+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|s1     |Movie  |Dick Johnson Is Dead           

In [35]:
netflix_df.summary().show()

+-------+----------------+-------------+---------------------------------+------------+--------+----------+------------------+--------------------+--------+------------------+
|summary|         show_id|         type|                            title|    director| country|date_added|      release_year|              rating|duration|         listed_in|
+-------+----------------+-------------+---------------------------------+------------+--------+----------+------------------+--------------------+--------+------------------+
|  count|            8791|         8791|                             8791|        8790|    8790|      8790|              8789|                8790|    8789|              8789|
|   mean|            NULL|         NULL|               1124.7692307692307|        NULL|  1944.0|      NULL|2014.1911480259416|                NULL|    NULL|              NULL|
| stddev|            NULL|         NULL|               1042.1800991068478|        NULL|    NULL|      NULL|  8.794154289

In [40]:
# Null count / column
from pyspark.sql.functions import sum 

netflix_df.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in netflix_df.columns
]).show()


+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|show_id|type|title|director|country|date_added|release_year|rating|duration|listed_in|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|      0|   0|    0|       1|      1|         1|           2|     1|       2|        2|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+



In [43]:
# Clean from our own method

netflix_clean = SparkUtils.clean_df(netflix_df)

print("Null counts after cleaning:")
netflix_clean.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in netflix_clean.columns
]).show()

Null counts after cleaning:
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|show_id|type|title|director|country|date_added|release_year|rating|duration|listed_in|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+
|      0|   0|    0|       0|      0|         0|           0|     0|       0|        0|
+-------+----+-----+--------+-------+----------+------------+------+--------+---------+



In [44]:
# Write 
SparkUtils.write_df(netflix_clean, "release_year", "netflix_output")


In [45]:
# Validation
!ls /home/jovyan/notebooks/data/netflix_output/ | wc -l

# Example of validation
df_1990 = spark.read.parquet("/home/jovyan/notebooks/data/netflix_output").filter("release_year = 1990")
df_1990.show()

75


+-------+-------+--------------------+-------------------+-------------+----------+------+--------+--------------------+------------+
|show_id|   type|               title|           director|      country|date_added|rating|duration|           listed_in|release_year|
+-------+-------+--------------------+-------------------+-------------+----------+------+--------+--------------------+------------+
|   s145|  Movie|         House Party|    Reginald Hudlin|United States|  9/1/2021|     R| 104 min|Comedies, Cult Mo...|        1990|
|  s1252|  Movie|  Dances with Wolves|      Kevin Costner|United States|  3/1/2021| PG-13| 181 min|Action & Adventur...|        1990|
|  s1622|  Movie|  Quigley Down Under|       Simon Wincer|    Australia| 12/1/2020| PG-13| 120 min|  Action & Adventure|        1990|
|  s1862|  Movie|                 Dil|        Indra Kumar|        India|10/12/2020| TV-14| 165 min|Comedies, Dramas,...|        1990|
|  s2318|  Movie|        Total Recall|     Paul Verhoeven|Unit

In [ ]:
sc.stop()